In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amazonreviews/train.ft.txt.bz2
/kaggle/input/amazonreviews/test.ft.txt.bz2


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import bz2

from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re
import csv
import nltk

from keras.layers import *
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

%matplotlib inline


import os
print(os.listdir("../input"))


['amazonreviews']


Using TensorFlow backend.


In [7]:
# import train data
data = bz2.BZ2File('../input/amazonreviews/train.ft.txt.bz2')
data = data.readlines()
data = [x.decode('utf-8') for x in data]
print(len(data))

3600000


In [8]:
pip install keras-self-attention


Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install fasttext


Note: you may need to restart the kernel to use updated packages.


In [10]:
from keras.models import Sequential
from keras_self_attention import SeqSelfAttention
import fasttext


In [11]:
help(fasttext.train_supervised)


Help on function train_supervised in module fasttext.FastText:

train_supervised(*kargs, **kwargs)
    Train a supervised model and return a model object.
    
    input must be a filepath. The input text does not need to be tokenized
    as per the tokenize function, but it must be preprocessed and encoded
    as UTF-8. You might want to consult standard preprocessing scripts such
    as tokenizer.perl mentioned here: http://www.statmt.org/wmt07/baseline.html
    
    The input file must must contain at least one label per line. For an
    example consult the example datasets which are part of the fastText
    repository such as the dataset pulled by classification-example.sh.



In [12]:
# Model Creation
model = fasttext.train_supervised('train.txt', label_prefix='__label__',epoch = 10)
print(model.labels)

['__label__1', '__label__2']


In [14]:
#Test data
test = bz2.BZ2File('../input/amazonreviews/test.ft.txt.bz2')
test = test.readlines()
test = [x.decode('utf-8') for x in test]


# To run the predict function, we need to remove the _label_1 and _label_2 from the testset.  
test_new = [w.replace('__label__2 ', '') for w in test]
test_new = [w.replace('__label__1 ', '') for w in test_new]
test_new = [w.replace('\n', '') for w in test_new]

# Use the predict function 
pred = model.predict(test_new)

# check the first record outputs
print(pred[0][0], 'is the predicted label')
print(pred[0][1], 'is the probability score')

['__label__2'] is the predicted label
['__label__2'] is the probability score


In [17]:
# recoding for 1's and 0's from test set and the actual predictions  
labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in test]
pred_labels = [0 if x == ['__label__1'] else 1 for x in pred[0]]

# run the accuracy measure. 
print(roc_auc_score(labels, pred_labels))

0.9172450000000001
